# IPL Data Analysis

## Match wins by teams

In [78]:
SELECT T.name AS Team
, COUNT(M.match_id) AS Wins
FROM dimteam T
LEFT JOIN dimmatch M ON T.team_id = M.winner_team_id
GROUP BY T.name
ORDER BY Wins DESC;

## Championships by team

In [79]:
SELECT T.name AS Team
, COUNT(M.match_id) AS Wins
FROM dimteam T
INNER JOIN dimmatch M ON T.team_id = M.winner_team_id AND M.stage = 'Final'
GROUP BY T.name
ORDER BY Wins DESC;

## Matches Played, Wins and Championships

In [80]:
WITH MatchesPlayed AS (
    SELECT DISTINCT team_id, match_id 
    FROM dimteam T
    INNER JOIN factinnings F ON T.team_id = F.bat_team_id
    UNION
    SELECT DISTINCT team_id, match_id 
    FROM dimteam T
    INNER JOIN factinnings F ON T.team_id = F.bat_team_id
)
SELECT T.name AS Team
, COUNT(M1.match_id) AS MatchesPlayed
, COUNT(M.match_id) AS Wins
, ROUND((CAST(COUNT(M.match_id) AS FLOAT)/COUNT(M1.match_id))*100, 2) AS [Win%]
, COUNT(C.match_id) AS Championships
FROM dimteam T
INNER JOIN MatchesPlayed M1 ON M1.team_id = T.team_id
LEFT JOIN dimmatch M ON T.team_id = M.winner_team_id AND M.match_id = M1.match_id
LEFT JOIN dimmatch C ON T.team_id = C.winner_team_id AND C.match_id = M1.match_id AND C.stage = 'Final'
GROUP BY T.name
ORDER BY Wins DESC;

In [81]:
SELECT MAX(date) FROM dimmatch;

In [82]:
SELECT * FROM dimmatch WHERE date = '2025-05-08';

## Balls taken to hit a Six by Player (Min 50 sixes)

In [83]:
SELECT P.player_name
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, SUM(CASE WHEN bat_runs = 6 THEN 1 ELSE 0 END) AS Sixes
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs = 6 THEN 1 END)) AS BallsPerSix 
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
GROUP BY P.player_name
HAVING SUM(CASE WHEN bat_runs = 6 THEN 1 ELSE 0 END) >= 50
ORDER BY BallsPerSix ASC, Sixes DESC;

## Balls taken for boundary (Min 50 boundaries)

In [84]:
SELECT P.player_name
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
GROUP BY P.player_name
HAVING SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) >= 50
ORDER BY BallsPerBoundary ASC, Boundary DESC;

## Balls taken for boundary in powerplay (Min 50 boundaries)

In [85]:
SELECT P.player_name
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
WHERE F.[over] BETWEEN 0 AND 5
GROUP BY P.player_name
HAVING SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) >= 50
ORDER BY BallsPerBoundary ASC, Boundary DESC;

## Ipl 2025 Top 10 Run Getters

In [86]:
SELECT TOP 10 P.player_name
, SUM(I.bat_runs) AS Runs
FROM factinnings I
INNER JOIN dimplayer P ON I.batter_id = P.player_id
INNER JOIN dimmatch M ON M.match_id = I.match_id
WHERE YEAR(M.[date]) = 2025
GROUP BY P.player_name
ORDER BY Runs DESC;

In [87]:
SELECT P.player_name
, SUM(I.bat_runs) AS Runs
, M.[date]
, M.team1_id
, M.team2_id
FROM factinnings I
INNER JOIN dimplayer P ON I.batter_id = P.player_id
INNER JOIN dimmatch M ON M.match_id = I.match_id
WHERE YEAR(M.[date]) = 2025
AND P.player_name = 'KL Rahul'
GROUP BY P.player_name, M.[date], M.team1_id, M.team2_id
ORDER BY M.[date] DESC;

In [88]:
SELECT 1, 'Powerplay' 
, P.player_name
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
WHERE F.[over] BETWEEN 0 AND 5
--AND P.player_name IN ('TM Head', 'Abhishek Sharma', 'Ishan Kishan')
GROUP BY P.player_name
HAVING SUM(F.bat_runs) >= 500
UNION
SELECT 2, 'Middle Overs' 
, P.player_name
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
WHERE F.[over] BETWEEN 6 AND 14
--AND P.player_name IN ('TM Head', 'Abhishek Sharma', 'Ishan Kishan')
GROUP BY P.player_name
HAVING SUM(F.bat_runs) >= 500
UNION
SELECT 3, 'Death' 
, P.player_name
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
WHERE F.[over] BETWEEN 15 AND 19
--AND P.player_name IN ('TM Head', 'Abhishek Sharma', 'Ishan Kishan')
GROUP BY P.player_name
HAVING SUM(F.bat_runs) >= 500
ORDER BY 1, BallsPerBoundary ASC, Boundary DESC;

In [89]:
SELECT *
FROM dimplayer
WHERE player_name LIKE '%Kishan%'

In [3]:
SELECT P.player_name
, M.season
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
INNER JOIN dimmatch M ON M.match_id = F.match_id
WHERE F.[over] BETWEEN 0 AND 5
AND P.player_name IN ('KL Rahul')
GROUP BY P.player_name, M.season
ORDER BY M.season DESC;

In [15]:
SELECT T.name AS [vs Team]
, M.venue
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
, M.[date] AS MatchDate
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
INNER JOIN dimmatch M ON M.match_id = F.match_id
INNER JOIN dimteam T ON T.team_id = F.bowl_team_id
WHERE P.player_name IN ('V Kohli')
AND M.season = '2025'
GROUP BY T.name, M.[date], M.match_number, M.venue
ORDER BY Runs DESC;

In [26]:
SELECT P.player_name AS Batter 
, B.player_name AS [vs Bowler]
, T.name AS [vs Team]
, M.venue
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
, M.[date] AS MatchDate
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
INNER JOIN dimmatch M ON M.match_id = F.match_id
INNER JOIN dimteam BT ON BT.team_id = F.bat_team_id
INNER JOIN dimteam T ON T.team_id = F.bowl_team_id
INNER JOIN dimplayer B ON F.bowler_id = B.player_id
WHERE M.season = '2025'
AND BT.abbr IN ('RCB', 'PBKS')
AND T.abbr IN ('RCB', 'PBKS')
GROUP BY P.player_name, B.player_name, T.name, M.[date], M.match_number, M.venue
ORDER BY Runs DESC;

In [20]:
SELECT B.bowling_style AS [vs Bowler Style]
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
INNER JOIN dimmatch M ON M.match_id = F.match_id
INNER JOIN dimteam T ON T.team_id = F.bowl_team_id
INNER JOIN dimplayer B ON F.bowler_id = B.player_id
WHERE P.player_name IN ('V Kohli')
AND M.season = '2025'
GROUP BY B.bowling_style
ORDER BY Runs DESC;

In [34]:
WITH Outs AS(
    SELECT P.player_id AS batter_id
    , CASE WHEN D.type IN ('retired out', 'retired hurt', 'run out', 'obstructing the field') THEN NULL 
        ELSE B.player_name END AS Bowler 
    , D.type AS Dismissal
    FROM factinnings F
    INNER JOIN dimplayer P ON F.player_out_id = P.player_id
    INNER JOIN dimmatch M ON M.match_id = F.match_id
    INNER JOIN dimteam BT ON BT.team_id = F.bat_team_id
    INNER JOIN dimteam T ON T.team_id = F.bowl_team_id
    INNER JOIN dimplayer B ON F.bowler_id = B.player_id
    INNER JOIN dimdismissal D ON D.dismissal_id = F.dismissal_id
    WHERE M.season = '2025'
    AND BT.abbr IN ('RCB', 'PBKS')
    AND T.abbr IN ('RCB', 'PBKS') 
)
SELECT P.player_name AS Batter 
, T.name AS [vs Team]
, M.venue
, SUM(F.bat_runs) AS Runs
, SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END) AS BallsFaced
, ROUND(CAST(SUM(F.bat_runs) AS FLOAT)*100/SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END), 2)AS SR
, O.Dismissal
, O.Bowler
, SUM(CASE WHEN bat_runs >= 4 THEN 1 ELSE 0 END) AS Boundary
, (SUM(CASE WHEN wides IS NULL THEN 1 ELSE 0 END)/SUM(CASE WHEN bat_runs >= 4 THEN 1 END)) AS BallsPerBoundary
, M.[date] AS MatchDate
FROM factinnings F
INNER JOIN dimplayer P ON F.batter_id = P.player_id
INNER JOIN dimmatch M ON M.match_id = F.match_id
INNER JOIN dimteam BT ON BT.team_id = F.bat_team_id
INNER JOIN dimteam T ON T.team_id = F.bowl_team_id
INNER JOIN dimplayer B ON F.bowler_id = B.player_id
LEFT JOIN Outs O ON O.batter_id = P.player_id
WHERE M.season = '2025'
AND BT.abbr IN ('RCB', 'PBKS')
AND T.abbr IN ('RCB', 'PBKS')
GROUP BY P.player_name, T.name, M.[date], M.match_number, M.venue, O.Dismissal, O.Bowler
ORDER BY Runs DESC;

In [32]:
SELECT P.player_id AS batter_id
    , P.player_name AS Batter
    , CASE WHEN D.type IN ('retired out', 'retired hurt', 'run out', 'obstructing the field') THEN NULL ELSE B.player_name END AS Bowler 
    , D.type AS Dismissal
    FROM factinnings F
    INNER JOIN dimplayer P ON F.player_out_id = P.player_id
    INNER JOIN dimmatch M ON M.match_id = F.match_id
    INNER JOIN dimteam BT ON BT.team_id = F.bat_team_id
    INNER JOIN dimteam T ON T.team_id = F.bowl_team_id
    INNER JOIN dimplayer B ON F.bowler_id = B.player_id
    INNER JOIN dimdismissal D ON D.dismissal_id = F.dismissal_id
    WHERE M.season = '2025'
    AND BT.abbr IN ('RCB', 'PBKS')
    AND T.abbr IN ('RCB', 'PBKS') 

In [31]:
SELECT *
FROM dimdismissal

In [69]:
SELECT B.player_name AS Bowler
, M.venue
, SUM(CASE WHEN D.type IN ('retired out', 'retired hurt', 'run out', 'obstructing the field') THEN 0 
        WHEN D.type IS NOT NULL THEN 1 
        ELSE 0 END) AS Wickets 
, (SUM(ISNULL(F.bat_runs, 0)) + SUM(ISNULL(F.wides, 0)) + SUM(ISNULL(F.noballs, 0))) AS RunsConceded        
, SUM(CASE WHEN F.wides > 0 OR F.noballs > 0 THEN 0 
        ELSE 1 END) AS Balls
, ROUND(CAST(SUM(ISNULL(F.bat_runs, 0)) + SUM(ISNULL(F.wides, 0)) AS FLOAT)
    /(CASE WHEN SUM(CASE WHEN D.type IN ('retired out', 'retired hurt', 'run out', 'obstructing the field') THEN NULL 
        WHEN D.type IS NOT NULL THEN 1 END) > 0 
        THEN SUM(CASE WHEN D.type IN ('retired out', 'retired hurt', 'run out', 'obstructing the field') THEN 0 
                    WHEN D.type IS NOT NULL THEN 1 
                    ELSE 0 END) END), 2) AS BowlAverage        
, ROUND((CAST(SUM(ISNULL(F.bat_runs, 0)) + SUM(ISNULL(F.wides, 0)) AS FLOAT)
    /SUM(CASE WHEN F.wides > 0 OR F.noballs > 0 THEN 0 ELSE 1 END))*6, 2) AS Economy                
FROM factinnings F
INNER JOIN dimmatch M ON M.match_id = F.match_id
INNER JOIN dimteam T ON T.team_id = F.bowl_team_id
INNER JOIN dimplayer B ON F.bowler_id = B.player_id
LEFT JOIN dimdismissal D ON D.dismissal_id = F.dismissal_id
WHERE M.season = '2025'
AND M.venue IN ('Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur', 'Narendra Modi Stadium, Ahmedabad')
GROUP BY B.player_name, M.venue
ORDER BY BowlAverage ASC;

In [35]:
SELECT DISTINCT venue
FROM dimmatch
WHERE YEAR([date]) = 2025